In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../src/')

In [2]:
import glob
import os
import pandas as pd
import sleepetl
from plotutils import gen_cal_plot_df, plotcal
from bokeh.io import show, output_notebook
from datetime import datetime
import json
import wodupcrawler

output_notebook()

Loading BokehJS ...

In [3]:
df_sleep = sleepetl.read_sleep_plot_df()
with open(f'{wodupcrawler.datadir}session_wods.json') as json_file:
    wods = json.load(json_file)

date = []
cals = []
wods_text = []
sleep = []

df_sleep['dt'] = df_sleep['start'].dt.strftime('%Y-%m-%d')

for f in glob.glob('/Users/hasannagib/Documents/s3stage/wahoo/heartrate_ts/*.csv'):
    dt = pd.to_datetime(os.path.basename(f)[:10])
    date.append(dt)
    cals.append(max(pd.read_csv(f)['calories']))
    try:
        wods_text.append(wods[dt.strftime('%Y-%m-%d')])
    except KeyError:
        wods_text.append([''] * 4)

    try:
        dur = df_sleep.set_index('dt').loc[dt.strftime('%Y-%m-%d')]['duration'] - \
              df_sleep.set_index('dt').loc['2020-10-30']['awake']
        sleep.append(dur)
    except KeyError:
        sleep.append(0)

In [4]:
pcal_30_df = gen_cal_plot_df(date, cals, wods_text, sleep, datetime.today() - pd.Timedelta('31 d'), datetime.today())
pcal_30, pcal_30_cds = plotcal(pcal_30_df)
show(pcal_30)

In [5]:
pcal_df = gen_cal_plot_df(date, cals, wods_text, sleep, datetime.today() - pd.Timedelta('165 d'), datetime.today())
pcal, pcal_cds = plotcal(
    pcal_df,
    weekdays=['Sat', 'Fri', 'Thu', 'Wed', 'Tue', 'Mon', 'Sun'],
    mode='github',
    fig_args={
        'plot_width':665,
        'plot_height':175,
        'tools':'hover',
        'toolbar_location':None,
    },
    hover_tooltips=[
        ("Date", "@Date"),
        ("Calories", f"@Cals"),
        ("Hours slept", "@sleep_hr:@sleep_min")
    ],
    show_dates=False
)

show(pcal)